In [4]:
from collections import defaultdict, deque
import heapq

# Sample repo dependency data (child -> parents)
dependencies = {
    "repoA": ["repoB", "repoC"],
    "repoB": ["repoC"],
    "repoC": [],
    "repoD": ["repoA", "repoB"],
    "repoE": ["repoD"],
    "repoF": ["repoE"]
}

# Build full descendant map (child + grandchildren + etc.)
reverse_deps = defaultdict(list)
for child, parents in dependencies.items():
    for parent in parents:
        reverse_deps[parent].append(child)

def get_all_descendants(repo):
    """BFS to find all repos that ultimately depend on this repo"""
    visited = set()
    queue = deque([repo])
    
    while queue:
        current = queue.popleft()
        for child in reverse_deps.get(current, []):
            if child not in visited:
                visited.add(child)
                queue.append(child)
    return visited

# Build repo metadata with full descendant counts
repo_metadata = {}
for repo in dependencies:
    num_parents = len(dependencies[repo])
    descendants = get_all_descendants(repo)
    num_descendants = len(descendants)
    
    repo_metadata[repo] = {
        "num_parents": num_parents,
        "num_descendants": num_descendants,
        "descendants": descendants
    }

# Filter out repos without any descendants
filtered_repos = {k:v for k,v in repo_metadata.items() if v["num_descendants"] > 0}

# Natural priority ratio without artificial scaling
def calculate_priority(repo_data):
    """Prioritize repos with many descendants and few dependencies"""
    return repo_data["num_descendants"] / (repo_data["num_parents"] + 1)

# Create priority queue using max-heap
priority_queue = []
for repo, data in filtered_repos.items():
    heapq.heappush(priority_queue, (-calculate_priority(data), repo))  # Negative for max-heap

# Sampling order
print("Enhanced Sampling Order:")
while priority_queue:
    score, repo = heapq.heappop(priority_queue)
    data = filtered_repos[repo]
    print(f"{repo}: {data['num_descendants']} descendants | {data['num_parents']} parents | Score: {-score:.2f}")

Enhanced Sampling Order:
repoC: 5 descendants | 0 parents | Score: 5.00
repoB: 4 descendants | 1 parents | Score: 2.00
repoA: 3 descendants | 2 parents | Score: 1.00
repoD: 2 descendants | 2 parents | Score: 0.67
repoE: 1 descendants | 1 parents | Score: 0.50
